In [1]:
from torch.utils.data import  Dataset
import polars 
import torch
from streaming.base import MDSWriter
from shutil import rmtree

In [2]:
train_df = polars.read_parquet("data/train_data.parquet")

In [3]:
feature_cols = ["symbol_id", "time_id"] +  [f"feature_{idx:02d}" for idx in range(79)]+ [f"responder_{idx}_lag_1" for idx in range(9)]

#select target values
target_cols = ["responder_6"]

# select the weight values
weight_cols = ["weight"]

In [4]:
import numpy as np

class CustomDataset(Dataset):
    def __init__(self, dataframe, accerlerator):
        # Store each part of the dataframe that needed as a tensor
        self.features = torch.FloatTensor(dataframe.select(feature_cols).to_numpy()).to(accerlerator)
        self.labels = torch.FloatTensor(dataframe.select(target_cols).to_numpy()).to(accerlerator)
        self.weights = torch.FloatTensor(dataframe.select(weight_cols).to_numpy()).to(accerlerator)


    def __len__(self):
        # Returb the length of the dataset
        return len(self.labels)

    def __getitem__(self, idx):
        # return the data at a given index
        # x = all features defined in the feature_cols
        # y = the target values that needed to be predicted definded in the target_cols
        # w = the weight values for calculating the loss defined in the weight_cols
        x = self.features[idx]
        y = self.labels[idx]
        w = self.weights[idx]
        return x, y, w

In [5]:
output_dir = 'data_for_nn/'
columns = {'x': 'pkl', 'y': 'pkl', 'w':'pkl'}
compression = 'zstd:7' # compress shards with ZStandard, level 7
# Here we use a human-readable string, but we could also
# pass in an int specifying the number of bytes.
limit = '100kb'
hashes = ['sha1'] # Use only SHA1 hashing on each shard

In [6]:
dataset = CustomDataset(train_df, accerlerator="cuda")

In [ ]:
del train_df

In [ ]:
with MDSWriter(out=output_dir, columns=columns, compression=compression, hashes=hashes, size_limit=limit) as out:
    for x, y , w in dataset:
        out.write({'x': x, 'y': y, "w":w })
        # raise error i think due to the data is too big